# Experimentación para crear las funciones requeridas para la API

En esta Jupyter Notebook se desarrollan y prueban en un primer momento las funciones que son solicitadas para la API. Luego, al momento del deployarlas es posible que sufran algunos cambios en función de los requerimientos propios de FastAPI o Render.

## Importaciones

In [1]:
import pandas as pd
from datetime import datetime

%load_ext autoreload
%autoreload 2
import utils

import warnings
warnings.filterwarnings("ignore")

## Carga de los dataset necesarios

Se cargan los conjuntos de datos necesarios para llevar a cabo las consultas solicitadas. Se decide subir los datos de esta manera, y no los dataser completos, debido a la limitada capacidad de almacenamiento que permite Render. Por otra parte, considerando que el objetivo es entregar un PVM, a modo de prueba de concepto, se consideró adecuado.

Los  dataset que se utilizan son:

* **df_reviews**: contiene la información relacionada a los usuarios que realizan reviews de los juegos en formato de análisis de sentimientos. Entre esta información, se encuentran las recomendaciones o no por parte de usuario, el nombre del juego en la que hizo una review, el desarrollador del mismo así como datos del usuario como su id, su url del perfil.
* **genre_ranking**: contiene el ranking de los géneros de juegos con mas horas jugadas.
* **df_items_developer**: contiene la información relacionada con cada item de juegos como su id, precio, desarrollador y año de lanzamiento.

In [2]:
df_reviews = pd.read_csv('data/df_reviews_unido.csv')

In [3]:
df_reviews

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_date,sentiment_analysis,items_count,price,genres,release_anio,developer,playtime_horas
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,2011-11-05,1,277.0,19.99,Action,2009,Tripwire Interactive,5060.666667
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,2014-06-24,1,888.0,NaN,Sin dato disponible,Sin dato disponible,Sin dato disponible,5539.916667
2,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,Formato inválido,2,137.0,9.99,Action,2013,"Hopoo Games, LLC",6180.266667
3,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,Formato inválido,2,137.0,9.99,Indie,2013,"Hopoo Games, LLC",6180.266667
4,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,Formato inválido,2,137.0,9.99,RPG,2013,"Hopoo Games, LLC",6180.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130628,lifeonhigh,http://steamcommunity.com/id/lifeonhigh,304050,No ratings yet,True,2015-12-20,1,45.0,0.00,Casual,2015,Trion Worlds,1862.433333
130629,lifeonhigh,http://steamcommunity.com/id/lifeonhigh,304050,No ratings yet,True,2015-12-20,1,45.0,0.00,Free to Play,2015,Trion Worlds,1862.433333
130630,lifeonhigh,http://steamcommunity.com/id/lifeonhigh,304050,No ratings yet,True,2015-12-20,1,45.0,0.00,Massively Multiplayer,2015,Trion Worlds,1862.433333
130631,lifeonhigh,http://steamcommunity.com/id/lifeonhigh,304050,No ratings yet,True,2015-12-20,1,45.0,0.00,RPG,2015,Trion Worlds,1862.433333


In [5]:
df_gastos_items = pd.read_csv('data/df_gastos_items_unido.csv')

In [90]:
genre_ranking = pd.read_csv('data/df_genre_ranking_unido.csv')

In [82]:
df_playtime_forever = pd.read_csv('data/df_playtime_forever_unido.csv')

In [95]:
df_items_developer = pd.read_csv('data/df_items_developer_unido.csv')

In [41]:
df_items_developer

,item_id,price,developer,release_anio
0,10,9.99,Valve,2000
1,20,4.99,Valve,1999
2,30,4.99,Valve,2003
3,40,4.99,Valve,2001
4,50,4.99,Gearbox Software,1999
...,...,...,...,...
10973,375450,59.99,"KOEI TECMO GAMES CO., LTD.",2016
10974,353390,NaN,Sin dato disponible,Sin dato disponible
10975,354280,5.99,CashcowGames,2016
10976,433920,14.99,"Amaranth Games, LLC",2016


In [9]:
utils.verificar_tipo_datos(df_items_developer)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,item_id,[<class 'int'>],100.0,0.0,0
1,developer,[<class 'str'>],100.0,0.0,0
2,release_anio,[<class 'str'>],100.0,0.0,0


## userdata

Esta función tiene por parámentro 'user_id' y debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendaciónes que realizó y cantidad de items.

In [30]:
def userdata(user_id):
    
    # Filtra por el usuario de interés
    usuario = df_reviews[df_reviews['user_id'] == user_id]
    # Calcula la cantidad de dinero gastado para el usuario de interés
    cantidad_dinero = df_gastos_items[df_gastos_items['user_id']== user_id]['price'].iloc[0]
    # Busca el count_item para el usuario de interés    
    count_items = df_gastos_items[df_gastos_items['user_id']== user_id]['items_count'].iloc[0]
    
    # Calcula el total de recomendaciones realizadas por el usuario de interés
    total_recomendaciones = usuario['reviews_recommend'].sum()
    # Calcula el total de reviews realizada por todos los usuarios
    total_reviews = len(df_reviews['user_id'].unique())
    # Calcula el porcentaje de recomendaciones realizadas por el usuario de interés
    porcentaje_recomendaciones = (total_recomendaciones / total_reviews) * 100
    
    return {
        'cantidad_dinero': cantidad_dinero,
        'porcentaje_recomendacion': round(porcentaje_recomendaciones, 2),
        'total_items': count_items.astype(int)
    }

In [31]:
user_id = 'EchoXSilence'
userdata(user_id)

{'cantidad_dinero': 189.84,
 'porcentaje_recomendacion': 0.03,
 'total_items': 23}

## countreviews

En esta función se ingresan dos fechas entre las que se quiere hacer una consulta y devuelve la cantidad de usuarios que realizaron reviews entre dichas fechas y, el porcentaje de las recomendaciones positivas (True) que los mismos hicieron.

In [44]:
def countreviews(fecha_inicio, fecha_fin):
    
    # Filtra el dataframe entre las fechas de interés
    user_data_entre_fechas = df_reviews[(df_reviews['reviews_date'] >= fecha_inicio) & (df_reviews['reviews_date'] <= fecha_fin)]
    # Calcula la cantidad de usuarios que dieron reviews entre las fechas de interés
    total_usuarios = user_data_entre_fechas['user_id'].nunique()
    # Calcula el total de recomendaciones entre las fechas de interes (True + False)
    total_recomendacion = len(user_data_entre_fechas)
    # Calcula la cantidad de recomendaciones positivas que que hicieron entre las fechas de interés
    total_recomendaciones_True = user_data_entre_fechas['reviews_recommend'].sum()
    # Calcula el porcentaje de recomendación realizadas entre el total de usuarios
    porcentaje_recomendaciones = (total_recomendaciones_True / total_recomendacion) * 100
    
    return {
        'total_usuarios_reviews': total_usuarios,
        'porcentaje_recomendaciones': round(porcentaje_recomendaciones,2)
    }

In [45]:
countreviews('2011-11-05', '2012-12-24')

{'total_usuarios_reviews': 926, 'porcentaje_recomendaciones': 98.68}

## genre

Esta función recibe como parámetro un género de videojuego y devuelve el puesto en el que se encuentra dicho género sobre un ranking de los mismos analizado la cantidad de horas jugadas para cada uno.

In [91]:
def genre(genero):
    # Busca el ranking para el género de interés
    rank = genre_ranking[genre_ranking['genres'] == genero]['ranking'].iloc[0]
    return {
        'rank': rank
    }

In [92]:
genero = 'Simulation'
genre(genero)

{'rank': 5}

## userforgenre

Esta función recibe como parámetro el género de un videojuego y devuelve el top 5 de los usuarios con más horas de juego en el género ingresado, indicando el id del usuario y el url de su perfil.

In [88]:
def userforgenre(genero):
    # Filtra el dataframe por el género de interés
    data_por_genero = df_playtime_forever[df_playtime_forever['genres'] == genero]
    # Agrupa el dataframe filtrado por usuario y suma la cantidad de horas
    top_users = data_por_genero.groupby(['user_url', 'user_id'])['playtime_horas'].sum().nlargest(5).reset_index()
    
    # Se hace un diccionario vacío para guardar los datos que se necesitan
    top_users_dict = {}
    for index, row in top_users.iterrows():
        # User info recorre cada fila del top 5 y lo guarda en el diccionario
        user_info = {
            'user_id': row['user_id'],
            'user_url': row['user_url']
        }
        top_users_dict[index + 1] = user_info
    
    return top_users_dict

In [89]:
genero = 'Action'
userforgenre(genero)

{1: {'user_id': 'Sp3ctre', 'user_url': 'http://steamcommunity.com/id/Sp3ctre'},
 2: {'user_id': 'shinomegami',
  'user_url': 'http://steamcommunity.com/id/shinomegami'},
 3: {'user_id': 'REBAS_AS_F-T',
  'user_url': 'http://steamcommunity.com/id/REBAS_AS_F-T'},
 4: {'user_id': 'Terminally-Chill',
  'user_url': 'http://steamcommunity.com/id/Terminally-Chill'},
 5: {'user_id': 'DownSyndromeKid',
  'user_url': 'http://steamcommunity.com/id/DownSyndromeKid'}}

## developer

Esta función recibe como parámetro 'developer' que es la empresa desarrolladora del juego y devuelve la cantidad de items que desarrolla dicha empresa y el porcentaje de contenido Free por año según empresa desarrolladora.

In [98]:
df_items_developer.head(2)

,price,release_anio,developer,item_id
0,4.99,2018,Kotoshiro,761140
1,0.00,2018,Secret Level SRL,643980


In [99]:
def developer(desarrollador):
    # Filtra el dataframe por desarrollador de interés
    data_filtrada = df_items_developer[df_items_developer['developer'] == desarrollador]
    # Calcula la cantidad de items por año
    cantidad_por_año = data_filtrada.groupby('release_anio')['item_id'].count()
    # Calcula la cantidad de elementos gratis por año
    cantidad_gratis_por_año = data_filtrada[data_filtrada['price'] == 0.0].groupby('release_anio')['item_id'].count()
    # Calcula el porcentaje de elementos gratis por año
    porcentaje_gratis_por_año = (cantidad_gratis_por_año / cantidad_por_año * 100).fillna(0).astype(int)

    result_dict = {
        'cantidad_por_año': cantidad_por_año.to_dict(),
        'porcentaje_gratis_por_año': porcentaje_gratis_por_año.to_dict()
    }
    
    return result_dict

In [100]:
desarrollador = 'Mechanical Boss'
developer(desarrollador)

{'cantidad_por_año': {'2016': 2, '2017': 1},
 'porcentaje_gratis_por_año': {'2016': 50, '2017': 0}}

## sentiment_analysis

In [108]:
df_reviews.head(10)

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_date,sentiment_analysis,items_count,price,genres,release_anio,developer,playtime_horas
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,2011-11-05,1,277.0,19.99,Action,2009,Tripwire Interactive,5060.666667
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,2014-06-24,1,888.0,NaN,Sin dato disponible,Sin dato disponible,Sin dato disponible,5539.916667
2,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,Formato inválido,2,137.0,9.99,Action,2013,"Hopoo Games, LLC",6180.266667
3,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,Formato inválido,2,137.0,9.99,Indie,2013,"Hopoo Games, LLC",6180.266667
4,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,Formato inválido,2,137.0,9.99,RPG,2013,"Hopoo Games, LLC",6180.266667
5,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,2013-10-14,2,541.0,24.99,Adventure,2013,Telltale Games,7633.366667
6,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,2014-04-15,1,629.0,19.99,Action,2012,FromSoftware,14165.433333
7,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,2014-04-15,1,629.0,19.99,RPG,2012,FromSoftware,14165.433333
8,Wackky,http://steamcommunity.com/id/Wackky,249130,7 of 8 people (88%) found this review helpful,True,2014-05-05,0,NaN,19.99,Action,2013,Traveller's Tales,NaN
9,Wackky,http://steamcommunity.com/id/Wackky,249130,7 of 8 people (88%) found this review helpful,True,2014-05-05,0,NaN,19.99,Adventure,2013,Traveller's Tales,NaN


In [106]:
def sentiment_analysis(anio):
    # Filtrar las reseñas del desarrollador específico
    anio_reviews = df_reviews[df_reviews['release_anio'] == anio]
    
    # Inicializar un diccionario para contar las categorías de sentimiento
    sentiment_counts = {'Negative': 0, 'Neutral': 0, 'Positive': 0}
    
    # Iterar a través de las reseñas del desarrollador
    for index, row in anio_reviews.iterrows():
        sentiment = row['sentiment_analysis']
        sentiment_category = ''
        
        # Asignar la categoría de sentimiento correspondiente
        if sentiment == 0:
            sentiment_category = 'Negative'
        elif sentiment == 1:
            sentiment_category = 'Neutral'
        elif sentiment == 2:
            sentiment_category = 'Positive'
        
        # Incrementar el contador correspondiente en el diccionario
        sentiment_counts[sentiment_category] += 1
    
    return sentiment_counts

In [101]:
anio = 2000
sentiment_analysis(anio
                   )

{'Negative': 969, 'Neutral': 7783, 'Positive': 4817}